In [1]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [2]:
DATA_DIR = './library-data'

In [3]:
bks = pd.read_csv(f'{DATA_DIR}/books.csv')
ckts = pd.read_csv(f'{DATA_DIR}/checkouts.csv')
cmrs = pd.read_csv(f'{DATA_DIR}/customers.csv')
lbrs = pd.read_csv(f'{DATA_DIR}/libraries.csv')

In [4]:
bks.shape, ckts.shape, cmrs.shape, lbrs.shape

((240, 8), (2000, 5), (2000, 10), (18, 6))

In [5]:
bks.sample(3)

,id,title,authors,publisher,publishedDate,categories,price,pages
83,9HOEMS9ffGgC,Core Clinical Medicine,['Gordon W. Stewart'],World Scientific,2010,['Medical'],433.99,744
197,ZIRMAAAAMAAJ,The Steam Engine,['American School of Correspondence at Armour ...,NaN,1907,['Steam-engines'],428.5,762
136,z9-lHMQjvD0C,"Speeches, addresses and letters on industrial ...",['William Darrah KELLEY'],NaN,1872,NaN,58.99,358


In [6]:
ckts.sample(3)

,id,patron_id,library_id,date_checkout,date_returned
953,kdDDAgAAQBAJ,28483a0ae38ab558f6be37e16013837d,zzw-222@5xc-knn-c5z,2018-02-24,2018-03-03
1031,uW1BAQAAMAAJ,aaed0adb64c150af2cb029d9d2248f0c,23v-222@5xc-jv7-v4v,1805-08-17,2018-06-15
449,bbbFl0dt6WIC,fba24f65d03fe11f17bd37ed8a795648,zzw-223@5xc-jv7-ct9,2018-11-22,2018-12-04


In [7]:
len(set(ckts.id).intersection(set(bks.id))), bks.id.nunique()

(240, 240)

In [8]:
cmrs.sample(3)

,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
852,3ef3c7ce2784643320adc8bd98f53e94,Richard McKay,7719 N Westanna Ave,Portland,Oregon,97203.0,2000-08-19,male,College,Admin & Support
854,79aba093cc41da3ae041e263e19026e4,Delila Ritchie,5604 SE 57th Ave,Portland,NaN,97206.0,1976-03-10,female,Graduate Degree,Education & Health
666,fb0a668b335686b3489f1fd09e39997a,Joan Laberdee,59 Touchstone,Lake Oswego,Oregon,97035.0,1966-04-05,female,College,NaN


In [9]:
len(set(ckts.patron_id).intersection(set(cmrs.id))), cmrs.id.nunique()

(2000, 2000)

In [10]:
lbrs.sample(3)

,id,name,street_address,city,region,postal_code
15,222-222@5xc-jxp-rp9,Multnomah County Library Hollywood Library,4040 NE Tillamook St,Portland,OR,97212_
17,224-222@5xc-jw2-t9z,Multnomah COUNTY library Gregory Heights,7921 NE sandy BLVD,Portland,OR,_97213
3,227-222@5xc-jww-btv,Multnomah County Library Hillsdale,1525 SW Sunset blvd,Portland,or,-97239


In [11]:
len(set(ckts.library_id).intersection(set(lbrs.id))), lbrs.id.nunique()

(18, 18)

In [12]:
ckts[['date_checkout', 'date_returned']].describe()

,date_checkout,date_returned
count,1935,1942
unique,573,622
top,2018-01-04,2018-07-30
freq,10,13


In [13]:
ckts[['date_checkout', 'date_returned']].isna().sum()

date_checkout    65
date_returned    58
dtype: int64

In [14]:
# Many values have time inconsistencies

ckts.loc[0, ['date_returned']].values[0], ckts.loc[0, ['date_checkout']].values[0]

('2018-11-13', '2019-01-28')

In [15]:
## Filling missing values with current date avoids dropping the data, but also doesn't add outliers to the distribution
## Dates are filled with assumption that dataset are always historical from the moment of processing
## Time inconsistencies can still exist but will be tackled below

today = datetime.today()
one_month_later = today + relativedelta(months=1)
two_months_later = today + relativedelta(months=2)

one_month_later_str = one_month_later.strftime('%Y-%m-%d')
two_months_later_str = two_months_later.strftime('%Y-%m-%d')

ckts['date_checkout'] = ckts['date_checkout'].fillna(one_month_later_str)
ckts['date_returned'] = ckts['date_returned'].fillna(two_months_later_str)

print(ckts[['date_checkout', 'date_returned']].isna().sum())

date_checkout    0
date_returned    0
dtype: int64


In [16]:
ckts.shape

(2000, 5)

In [17]:
def clean_date_string(date_str):
    if pd.isna(date_str):
        return None

    date_str = str(date_str).strip()

    date_str = re.sub(r'^[^0-9A-Za-z]+|[^0-9A-Za-z]+$', '', date_str)

    if re.fullmatch(r'\d{8}', date_str):
        return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    
    date_str = re.sub(r'[^\w]', '-', date_str)  # e.g., ' ', '/', '|', etc.

    return date_str

ckts['date_checkout_cleaned_str'] = ckts['date_checkout'].apply(clean_date_string)

ckts['date_checkout_parsed'] = pd.to_datetime(
    ckts['date_checkout_cleaned_str'],
    errors='coerce',
    dayfirst=True
)

ckts['date_checkout_clean'] = ckts['date_checkout_parsed'].dt.strftime('%Y-%m-%d')
ckts['date_checkout_clean'].value_counts()


/var/folders/mn/wh8_vw8s4vx2lwxn82d99cnm0000gp/T/ipykernel_8151/2920434002.py:18: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ckts['date_checkout_parsed'] = pd.to_datetime(


date_checkout_clean
2025-05-21    65
2018-07-06    11
2018-12-12    10
2018-11-16    10
2018-04-07    10
              ..
2111-01-11     1
2117-05-19     1
1817-04-21     1
2119-12-27     1
1801-02-16     1
Name: count, Length: 544, dtype: int64

In [18]:
ckts['date_returned_cleaned_str'] = ckts['date_returned'].apply(clean_date_string)

ckts['date_returned_parsed'] = pd.to_datetime(
    ckts['date_returned_cleaned_str'],
    errors='coerce',
    dayfirst=True
)

ckts['date_returned_clean'] = ckts['date_returned_parsed'].dt.strftime('%Y-%m-%d')
ckts['date_returned_clean'].value_counts()

/var/folders/mn/wh8_vw8s4vx2lwxn82d99cnm0000gp/T/ipykernel_8151/3929197215.py:3: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ckts['date_returned_parsed'] = pd.to_datetime(


date_returned_clean
2025-06-21    58
2018-07-30    13
2018-06-01    12
2018-12-17    11
2018-06-27    11
              ..
2119-07-06     1
2112-09-06     1
2113-01-17     1
2019-02-06     1
2107-06-16     1
Name: count, Length: 588, dtype: int64

In [19]:
ckts['date_returned_clean'] = pd.to_datetime(ckts['date_returned_clean'], errors='coerce')
ckts['date_checkout_clean'] = pd.to_datetime(ckts['date_checkout_clean'], errors='coerce')


ckts['days_out'] = (ckts['date_returned_clean'] - ckts['date_checkout_clean']).dt.days
ckts['days_out'].value_counts()

days_out
 15       72
 8        69
 5        68
 6        67
 16       67
          ..
 31477     1
-31027     1
 79004     1
-2599      1
 32651     1
Name: count, Length: 605, dtype: int64

In [20]:
ckts.days_out.isna().sum()

0

In [21]:
fig = px.histogram(
    ckts,
    x='days_out',
    nbins=50,
    title='Distribution of Days Out',
    labels={'days_out': 'Days Out'}
)

fig.update_layout(bargap=0.1)
fig.show()

In [22]:
## Using IQR for skewed outlier detection
days_out = ckts['days_out']

Q1 = days_out.quantile(0.25)
Q3 = days_out.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

ckts['possible_error'] = (ckts['days_out'] < lower_bound) | (ckts['days_out'] > upper_bound)

print(ckts['possible_error'].value_counts())

fig = px.histogram(
    ckts,
    x='days_out',
    nbins=20,
    color='possible_error',
    title='Distribution of Days Out with IQR-Based Outlier Detection',
    labels={'days_out': 'Days Out'}
)

fig.add_vline(x=lower_bound, line_dash='dash', line_color='blue', annotation_text='Lower IQR Bound', annotation_position='bottom left')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='orange', annotation_text='Upper IQR Bound', annotation_position='top right')

fig.update_layout(bargap=0.1)
fig.show()

possible_error
False    1376
True      624
Name: count, dtype: int64


In [23]:
ckts[~ckts['possible_error']]['date_checkout_clean'].min(), ckts[~ckts['possible_error']]['date_returned_clean'].max()

(Timestamp('2018-01-01 00:00:00'), Timestamp('2019-02-06 00:00:00'))

In [24]:
ckts['possible_error'].value_counts()

possible_error
False    1376
True      624
Name: count, dtype: int64

In [25]:
error_df = ckts[ckts['possible_error']].copy()

error_df['checkout_year'] = error_df['date_checkout_clean'].dt.year
error_df['returned_year'] = error_df['date_returned_clean'].dt.year

melted = error_df.melt(
    id_vars='possible_error',
    value_vars=['checkout_year', 'returned_year'],
    var_name='date_type',
    value_name='year'
)

fig = px.histogram(
    melted,
    x='year',
    color='date_type',
    barmode='overlay',
    title='Year Distribution of Years with Possible Errors',
    labels={'year': 'Year'}
)

fig.show()

In [26]:
## auditing possible errors in dates

returned_year = ckts['date_returned_clean'].dt.year

def fix_future_year(date):
    if pd.isna(date):
        return date
    if date.year >= 2100 and str(date.year).startswith('21'):
        # replacing '21xx' with '20xx' by subtracting 100 years
        return date.replace(year=date.year - 100)
    return date

ckts['date_returned_cleanER'] = ckts['date_returned_clean'].apply(fix_future_year)
ckts['date_checkout_cleanER'] = ckts['date_checkout_clean'].apply(fix_future_year)


In [27]:
## if still checkout date is after returned date, swap these values

mask_swap = ckts['date_checkout_cleanER'] > ckts['date_returned_cleanER']
ckts.loc[mask_swap, ['date_checkout_cleanER', 'date_returned_cleanER']] = \
ckts.loc[mask_swap, ['date_returned_cleanER', 'date_checkout_cleanER']].values

In [28]:
ckts['days_out_final'] = (ckts['date_returned_cleanER'] - ckts['date_checkout_cleanER']).dt.days

fig = px.histogram(
    ckts,
    x='days_out_final',
    nbins=50,
    title='Distribution of updated Days Out',
    labels={'days_out_final': 'Days Out'}
)

fig.update_layout(bargap=0.1)
fig.show()

In [29]:
## Recalculating thresholds for outliers

days_out_final = ckts.days_out_final

Q1 = days_out_final.quantile(0.25)
Q3 = days_out_final.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

ckts['error'] = (ckts['days_out_final'] < lower_bound) | (ckts['days_out_final'] > upper_bound)

print(ckts['error'].value_counts())

fig = px.histogram(
    ckts,
    x='days_out_final',
    nbins=20,
    color='error',
    title='Distribution of updated Days Out with IQR-Based Outlier Detection',
    labels={'days_out_final': 'Days Out'}
)

fig.add_vline(x=lower_bound, line_dash='dash', line_color='blue', annotation_text='Lower IQR Bound', annotation_position='bottom left')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='orange', annotation_text='Upper IQR Bound', annotation_position='top right')

fig.update_layout(bargap=0.1)
fig.show()

error
False    1544
True      456
Name: count, dtype: int64


In [30]:
ckts.columns

Index(['id', 'patron_id', 'library_id', 'date_checkout', 'date_returned',
       'date_checkout_cleaned_str', 'date_checkout_parsed',
       'date_checkout_clean', 'date_returned_cleaned_str',
       'date_returned_parsed', 'date_returned_clean', 'days_out',
       'possible_error', 'date_returned_cleanER', 'date_checkout_cleanER',
       'days_out_final', 'error'],
      dtype='object')

In [31]:
ckts = ckts[ckts['error'] == False].reset_index(drop=True)
ckts = ckts[['id', 'patron_id' ,'library_id', 'date_checkout_cleanER', 'date_returned_cleanER', 'days_out_final']]
ckts.sample(3)

,id,patron_id,library_id,date_checkout_cleanER,date_returned_cleanER,days_out_final
186,QT21QytTCiUC,651fffb7a73f4df652ba22beb348d2fe,22d-222@5xc-kcy-8sq,2018-07-08,2018-07-23,15
916,e3XVAAAAMAAJ,e17a82364f09fff530c609d8ae9d8d83,zzw-223@5xc-jv7-ct9,2018-04-06,2018-10-19,196
1501,ZAtjZwZN5pcC,d06d692ccd7b11da6222e2fbcfa7fb7e,23v-222@5xc-jv7-v4v,2018-11-16,2018-11-29,13


In [32]:
ckts['return_status'] = ckts['days_out_final'].apply(
    lambda x: 'Late' if x > 28 else 'On Time'
)

ckts['is_late'] = ckts['return_status'].map({
    'Late': 1,
    'On Time': 0
})

ckts['is_late'].value_counts()

is_late
0    1383
1     161
Name: count, dtype: int64

In [33]:
ckts = ckts.rename(columns={'id':'checkout_id', 'patron_id':'customer_id'})
ckts = ckts[['checkout_id', 'customer_id', 'library_id', 'days_out_final', 'is_late']]
ckts.sample(3)

,checkout_id,customer_id,library_id,days_out_final,is_late
1497,P2FYAAAAMAAJ,59bc6c95558f76890e45ae513d4327f5,zzw-224@5xc-jwv-2rk,18,0
575,ThY8ZkU58rwC,2bc8b831cf0797d5d6a1a07c08b57931,227-222@5xc-jww-btv,19,0
1337,frzDCQAAQBAJ,cd5a1f2f2422acba84610f01e4545644,222-222@5xc-kkw-bzf,18,0


In [35]:
ckts.to_csv(f'{DATA_DIR}/checkouts_processed.csv', index=False)